In [40]:
# from pddl import parse_domain, parse_problem

# DOMAIN_PATH = 'pddl_files/domain.pddl'
# PROBLEM_PATH = 'pddl_files/problem0.pddl'

# domain = parse_domain(DOMAIN_PATH)
# problem = parse_problem(PROBLEM_PATH)

In [1]:
from parser_files.pddlparser import PDDLParser

parser = PDDLParser()
domain = parser.parse('pddl_files/domain.pddl')
problem = parser.parse('pddl_files/problem0.pddl')

In [2]:
objects = problem.initial_state.objects

for action in domain.actions:
    for param in zip(action.arg_names, action.types):
        print(param)

('?pos1', 'position')
('?pos2', 'position')
('?pos1', 'position')
('?pos2', 'position')


In [14]:
import itertools

def get_value_predicate(predicate, candidate):
    candidate_predicate = tuple([predicate[0]] + [candidate[i] for i in predicate[1:] if i])

    # if predicate has arity = 0
    if len(candidate_predicate) == 1:
        candidate_predicate = (predicate[0], '')
    return candidate_predicate


def get_actions(current_predicates, constant_predicates, all_actions):
    actions = []
    for action in all_actions:
        possible_values = {
            param[0]: objects[param[1]] for param in zip(action.arg_names, action.types)
        }
        for candidate in ( (dict(zip(possible_values, x)) for x in itertools.product(*possible_values.values()) )):
            action_is_valid = True
            for precond in action.preconditions.literals:
                # Negative predicate
                if precond[0]==-1:
                    neg_precond = precond[1]
                    # Applies candidate's values as arguments for the predicate
                    neg_candidate_precond = get_value_predicate(neg_precond, candidate)
                    if neg_candidate_precond in current_predicates or neg_candidate_precond in constant_predicates:
                        action_is_valid = False
                        break
                # Positive predicate
                else:
                    candidate_precond = get_value_predicate(precond, candidate)
                    if candidate_precond not in current_predicates and candidate_precond not in constant_predicates:
                        action_is_valid = False
                        break
            if action_is_valid:
                actions.append((action, candidate))
    return actions


def get_neighbors(current_state, constant_predicates, all_actions):
    # get all possible actions
    actions = get_actions(current_state, constant_predicates, all_actions)

    # compute all successor states
    states = []
    for action in actions:
        state = current_state.copy()
        action_function, candidate = action
        for predicate in action_function.effects.literals:
            if predicate[0]==-1:
                neg_value_predicate = get_value_predicate(predicate[1], candidate)
                if neg_value_predicate in state:
                    state.remove(neg_value_predicate)
                    state.append((-1, neg_value_predicate))
            else:
                value_predicate = get_value_predicate(predicate, candidate)
                if value_predicate not in state:
                    state.append(value_predicate)
        states.append(state)
    return states


constant_predicates, initial_predicates = [], []

for predicate in problem.initial_state.predicates:
    if predicate[0] in ["adjacent", "obstacle"]:
        constant_predicates.append(predicate)
    else:
        initial_predicates.append(predicate)

initial_predicates


states = get_neighbors(initial_predicates, constant_predicates, domain.actions)

(6,
 [[('munition', ''), ('at', 'pos52'), (-1, ('at', 'pos62'))],
  [('munition', ''), ('at', 'pos61'), (-1, ('at', 'pos62'))],
  [('munition', ''), ('at', 'pos63'), (-1, ('at', 'pos62'))],
  [('at', 'pos62'), (-1, ('munition', ''))],
  [('at', 'pos62'), (-1, ('munition', ''))],
  [('at', 'pos62'), (-1, ('munition', ''))]])